In [1]:
from scipy.stats import nbinom

def premier_ev(win_rate = 0.5, pack_value=200):
    rewards = [
        50 + 1 * pack_value,
        100 + 1 * pack_value,
        250 + 2 * pack_value,
        1000 + 2 * pack_value,
        1400 + 3 * pack_value,
        1600 + 4 * pack_value,
        1800 + 5 * pack_value,
        2200 + 6 * pack_value
    ]
    # games counts the number of games until we get 3 losses
    games = nbinom(n=3, p=1-win_rate)
    ev = games.expect(
        ub=7, # We can only win at most 7 games
        func=lambda x: rewards,
    ) + (games.sf(7)) * rewards[-1]
    # We have to add the probability of losing less than 3 times, still counts as 7 wins, hence this last term.
    # This saves us having to sum over the possibly infinite range of the RV
    cost = 1500
    return ev - cost

In [2]:
from scipy.stats import binom

def traditional_ev(win_rate = 0.5, pack_value=200):
    rewards = [
        1 * pack_value,
        1 * pack_value,
        1000 + 4 * pack_value,
        3000 + 6 * pack_value
    ]
    wins = binom(n=3, p=win_rate)
    ev = wins.expect(
        func=lambda x: rewards # The value of a win is looked up in the table
    )
    cost = 1500
    return ev - cost

In [3]:
from scipy.stats import nbinom
from numpy import fromiter
from itertools import islice

def quick_ev(win_rate = 0.5, pack_value=200):
    rewards = [
        50 + 1 * pack_value,
        100 + 1 * pack_value,
        200 + 1 * pack_value,
        300 + 1 * pack_value,
        450 + 1 * pack_value,
        650 + 1 * pack_value,
        850 + 1 * pack_value,
        950 + 2 * pack_value
    ]

    # games counts the number of wins until we get 3 losses
    games = nbinom(n=3, p=1-win_rate)
    ev = games.expect(
        ub=7,
        func=lambda x: rewards # The number of games-3 is the number of wins
    ) + (games.sf(7)) * rewards[-1]
    # We have to add the probability of losing less than 3 times, still counts as 7 wins, hence this last term.
    # This saves us having to sum over the possibly infinite range of the RV
    cost = 750
    return ev - cost

In [4]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot
from numpy import linspace, apply_along_axis, vectorize

init_notebook_mode(connected=True)

x = linspace(0, 1, 21)
    
fig = go.Figure(
    data=[
        go.Scatter(x=x, y=apply_along_axis(vectorize(quick_ev), 0, x, pack_value=0), name='Quick Draft (gems only)'),
        go.Scatter(x=x, y=apply_along_axis(vectorize(premier_ev), 0, x, pack_value=0), name='Premier Draft (gems only)'),
        go.Scatter(x=x, y=apply_along_axis(vectorize(traditional_ev), 0, x, pack_value=0), name='Traditional Draft (gems only)'),
        go.Scatter(x=x, y=apply_along_axis(vectorize(quick_ev), 0, x, pack_value=200), name='Quick Draft (packs worth 200)'),
        go.Scatter(x=x, y=apply_along_axis(vectorize(premier_ev), 0, x, pack_value=200), name='Premier Draft (packs worth 200)'),
        go.Scatter(x=x, y=apply_along_axis(vectorize(traditional_ev), 0, x, pack_value=200), name='Traditional Draft (packs worth 200)')
    ],
    layout=go.Layout(
        title="Expected Gem Reward",
        xaxis=dict(title='Winrate', range=[0, 1], dtick=0.1),
        yaxis=dict(title='Gem Gain')
   )
)
iplot(fig)